In [ ]:

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np

In [ ]:
#Fragen Aufgabenstellung:
        
    #Können wir aus den Bondaten irgendwelche Erkenntisse gewinnen? -> Unsupervised RFM Analyse
    #Gibt es Kuden, die sehr stark in bestimmten Warengruppen kaufen? -> EDA
    #Gibt es Kunden die immer nur zu bestimmten Tagen oder Monaten bei uns einkaufen? -> EDA
    

#Vorgehen
# 1. Hochladen und erste Anpassungen an den Daten
# 2. Verstehen der Daten
# 3. Vorbereitung der Daten 
# 4. EDA
    #4.1. Wochentage
    #4.2. Monate
    #4.3. Produktgruppen
# 5. Kundensegmentierung Analyse
#RFM Analysis:
    #1. Wann wurde der letzte Einkauf getätigt
    #2. Wie oft wurde etwas gekauft
    #3. Wie viel Geld wird im Durschnitt Ausgegeben

In [ ]:
#1.

#import of csv files 

bondaten = pd.read_csv("Bondaten.CSV", delimiter=';')

artikelstamm = pd.read_csv("Artikelstamm.csv", encoding='latin-1', delimiter=';')


In [ ]:
#2
#Join Bondaten/Artikelstamm zu Kundeninformationen


kundeninfo = bondaten.merge(artikelstamm, how='left', on ='ArtikelID')

kundeninfo.head()

#kundeninfo.describe()


# 1. Kaufdatum
# 2. Unique ID des Bon
# 3. Unique ID des Kunden
# 4. Unique ID des Artikels
# 5. Gekaufte Menge
# 6. Gesamt Netto Umsatz (Menge*Stückpreis -> Stückpreis = Umsatz/Menge)
# 7. Bereich des Artikels
# 8. Thema des Artikels
# 9. Unterthema des Artikels



In [ ]:
#Test ob es Produktgruppen gibt die nicht gekauft wurden

test = bondaten.merge(artikelstamm, how='right', on ='ArtikelID')


print(test.head())

print(test['Bereich'].isnull().values.any())

In [ ]:
#3.
#Basis unterstanding and transfomation of data

#Change data types

#date time
kundeninfo["Datum"] = pd.to_datetime(kundeninfo["Datum"], format ='%Y%m%d')

#IDs as object for better handeling
kundeninfo['BonID'] = kundeninfo['BonID'].astype(object)

kundeninfo['KundenID'] = kundeninfo['KundenID'].astype(object)

kundeninfo['ArtikelID'] = kundeninfo['ArtikelID'].astype(object)

#information about revene and quantity to float
kundeninfo['Umsatz'] = kundeninfo['Umsatz'].str.replace(',', '.').astype(float)

#kundeninfo['Umsatz'] = pd.to_numeric(kundeninfo['Umsatz'],errors='coerce')

kundeninfo['Menge'] = pd.to_numeric(kundeninfo['Menge'],errors='coerce')


kundeninfo['Monat'] = pd.DatetimeIndex(kundeninfo['Datum']).month

kundeninfo['Jahr'] = pd.DatetimeIndex(kundeninfo['Datum']).year

kundeninfo['Wochentag'] = pd.DatetimeIndex(kundeninfo['Datum']).dayofweek

#print(kundeninfo['Bereich'].unique())


#print(kundeninfo['Thema'].unique())

In [ ]:
#4.1. Wochentage 

umsatz_tag = verkaufinfo.groupby(['Wochentag','KundenID'], as_index = False)['Umsatz'].sum()

umsatz_tag = umsatz_tag.sort_values(['KundenID','Umsatz'])

#umsatz_monat['rank_Monat'] = umsatz_monat.groupby(['KundenID'], as_index = False)['Umsatz'].rank(method='min').astype(int)
umsatz_tag['rank_Tag'] = umsatz_tag.groupby(['KundenID'], as_index = False)['Umsatz'].cumcount(ascending=False) + 1


umsatz_tag_best = umsatz_tag.loc[umsatz_tag['rank_Tag'] == 1]

In [ ]:
   
#Summe bester Tag berücksichtigt  
sns.barplot(x='Wochentag'
             , y='Umsatz'
             , data = umsatz_tag_best
              ,estimator = np.sum
            ,errorbar=('ci', 0)
             )  
    

In [ ]:
sns.countplot(x='Wochentag', data=umsatz_tag_best)

In [ ]:

#Summe aller Umsätze berücksichtigt   
sns.barplot(x='Wochentag'
             , y='Umsatz'
             , data = umsatz_tag
              ,estimator = np.sum
            ,errorbar=('ci', 0)
            
             )  

In [ ]:
#4.2. Monat

umsatz_monat = verkaufinfo.groupby(['Monat','KundenID'], as_index = False)['Umsatz'].sum()

umsatz_monat = umsatz_monat.sort_values(['KundenID','Umsatz'])

umsatz_monat['rank_Monat'] = umsatz_monat.groupby(['KundenID'], as_index = False)['Umsatz'].cumcount(ascending=False) + 1


umsatz_monat_best = umsatz_monat.loc[umsatz_monat['rank_Monat'] == 1]

In [ ]:
#Summe bester Monat berücksichtigt  
sns.barplot(x='Monat'
             , y='Umsatz'
             , data = umsatz_monat_best
              ,estimator = np.sum
            ,errorbar=('ci', 0)
             )  


In [ ]:
sns.countplot(x='Monat', data=umsatz_monat_best)

In [ ]:
#Summe aller Umsätze berücksichtigt   
sns.barplot(x='Monat'
             , y='Umsatz'
             , data = umsatz_monat
              ,estimator = np.sum
            ,errorbar=('ci', 0)
            
             )  

In [ ]:
#4.3. Bereich, Thema, Unterthema


bereich_info = kundeninfo[['KundenID','Bereich','Umsatz']]

thema_info = kundeninfo[['KundenID','Thema','Umsatz']]

uthema_info = kundeninfo[['KundenID','Unterthema','Umsatz']]



umsatz_bereich = bereich_info.groupby(['KundenID','Bereich'], as_index = False)['Umsatz'].sum()

umsatz_thema = thema_info.groupby(['KundenID' ,'Thema'], as_index = False)['Umsatz'].sum()

umsatz_uthema = uthema_info.groupby(['KundenID','Unterthema'], as_index = False)['Umsatz'].sum()


umsatz_bereich = umsatz_bereich.sort_values(['KundenID','Umsatz'])
umsatz_bereich['rank_Bereich'] = umsatz_bereich.groupby(['KundenID'], as_index = False)['Umsatz'].cumcount(ascending=False) + 1

umsatz_bereich_best = umsatz_bereich.loc[umsatz_bereich['rank_Bereich'] == 1]


umsatz_thema = umsatz_thema.sort_values(['KundenID','Umsatz'])
umsatz_thema['rank_Thema'] = umsatz_thema.groupby(['KundenID'], as_index = False)['Umsatz'].cumcount(ascending=False) + 1

umsatz_thema_best = umsatz_thema.loc[umsatz_thema['rank_Thema'] == 1]


umsatz_uthema = umsatz_uthema.sort_values(['KundenID','Umsatz'])
umsatz_uthema['rank_Uthema'] = umsatz_uthema.groupby(['KundenID'], as_index = False)['Umsatz'].cumcount(ascending=False) + 1

umsatz_uthema_best = umsatz_uthema.loc[umsatz_uthema['rank_Uthema'] == 1]


print(umsatz_bereich.loc[umsatz_bereich['KundenID'] == 22392])

In [ ]:
sns.barplot(x='Bereich'
             , y='Umsatz'
             , data = umsatz_bereich
              ,estimator = np.sum
            ,errorbar=('ci', 0)
            
             )  

In [ ]:
sns.countplot(x='Bereich', data=umsatz_bereich_best)

In [ ]:
sns.countplot(x='Thema', data=umsatz_thema_best)

In [ ]:
sns.barplot(x='Unterthema'
             , y='Umsatz'
             , data = umsatz_uthema
              ,estimator = np.sum
            ,errorbar=('ci', 0)
             ) 

In [ ]:
sns.countplot(x='Unterthema', data=umsatz_uthema_best)

In [ ]:
#5. Kundensegmentierung mit K-Means 


##Zuerst auf eine BonID bringen

#Berechnung des letzten Einkaufs

kauf_info = kundeninfo.groupby(['Datum','KundenID', 'BonID'], as_index = False)['Umsatz'].sum()


kauf_info = kauf_info.sort_values(['KundenID','Datum'])
kauf_info['rank_Kauf'] = kauf_info.groupby(['KundenID'], as_index = False)['Datum'].cumcount(ascending=False) + 1

kauf_info_last = kauf_info.loc[kauf_info['rank_Kauf'] == 1]




kauf_info_last.loc['LetzterKauf'] = (pd.to_datetime(max(kauf_info_last['Datum'])) - kauf_info_last['Datum'] ).dt.days



In [ ]:
#Berechnung der Frequenz

freq = kauf_info.groupby('KundenID')['Datum'].count()

ki_freq = pd.DataFrame(freq).reset_index()
ki_freq.columns = ['KundenID','Frequenz']


last_freq = ki_freq.merge(kauf_info_last,on='KundenID')


print(last_freq.loc[last_freq['KundenID'] == 22392])

In [ ]:
#Berechnung des Einkaufswerts


m = kauf_info.groupby('KundenID', as_index = False)['Umsatz'].sum()

m.columns = ['KundenID','Umsatz_Gesamt']

print(m.loc[m['KundenID'] == 22392])

In [ ]:
#Data Frame zur Auswertung erstellen

rfm = m.merge(last_freq,on='KundenID')

finaldf = rfm[['KundenID','LetzterKauf','Frequenz','Umsatz_Gesamt']]

print(finaldf.loc[finaldf['KundenID'] == 22392])

In [ ]:
#Outliers löschen

import seaborn as sns
import matplotlib.pyplot as plt

list1 = ['LetzterKauf','Frequenz','Umsatz_Gesamt']
for i in list1:
    print(str(i)+': ')
    ax = sns.boxplot(x=finaldf[str(i)])
    plt.show()

In [ ]:
from sklearn.preprocessing import StandardScaler

new_df = new_df.drop_duplicates()
col_names = ['LetzterKauf','Frequenz','Umsatz_Gesamt']
features = new_df[col_names]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)
scaled_features = pd.DataFrame(features, columns = col_names)

scaled_features.head()

In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D

SSE = []
for cluster in range(1,10):
    kmeans = KMeans(n_clusters = cluster, init='k-means++')
    kmeans.fit(scaled_features)
    SSE.append(kmeans.inertia_)
    
# converting the results into a dataframe and plotting them
frame = pd.DataFrame({'Cluster':range(1,10), 'SSE':SSE})
plt.figure(figsize=(12,6))
plt.plot(frame['Cluster'], frame['SSE'], marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')

In [ ]:
# First, build a model with 3 clusters
kmeans = KMeans( n_clusters = 3, init='k-means++')
kmeans.fit(scaled_features)

In [ ]:
print(silhouette_score(scaled_features, kmeans.labels_, metric='euclidean'))

#Silhouette score:
    #3 Cluster: 0.54
    #4 Cluster: 0.50

    
# --> Choose 4 Cluster

In [ ]:
pred = kmeans.predict(scaled_features)
frame = pd.DataFrame(new_df)
frame['cluster'] = pred

In [ ]:
avg_df = frame.groupby(['cluster'], as_index=False).mean()
for i in list1:
    sns.barplot(x='cluster',y=str(i),data=avg_df)
    plt.show()

In [ ]:
fig = px.scatter_3d(data_frame=frame, x="LetzterKauf", 
                    y="Umsatz_Gesamt", z="Frequenz", color="cluster", height=550,
                   title = "Ergebnis der Kundensegmentierung")
fig.show()

In [ ]:
#Analyse aus KMeans

#Wie viele Pro Cluster -> was würden 10% in welchem Cluster mehr bringen

number = frame.groupby('cluster', as_index = False)['cluster'].count()


#Gesamt Umsatz pro Clust -> was würde 10% Umsatz steigerung bringen

umsatz = frame.groupby('cluster', as_index = False)['Umsatz_Gesamt'].sum()
